# Luke McMeans - Special Pop Lab

IMPORTANT NOTES:
- Due to issues with using AviationStack not properly accessing tail numbers, I choose to use AeroAPI.
- Taylor Swift's Plane Infomation isn't accessible through AeroAPI or pyflightdata since she requested to stop public tracking. It's not included in this notebook.
    - Unfortuantley, I couldn't find another celebrity who had publicly available flight data for this API or pyflightdata. I've instead added the tail numbers of planes I found on https://globe.adsbexchange.com/, along with a flight I recently took.

In [10]:
import pandas as pd
import requests

AeroAPI Key:

In [11]:
api_key = ""

In [12]:
# array for tail numbers we want to track
tail_numbers = [
    'N350XX',
    'N62892',
    'N474EP',
    'N2227Z',
    'N968RC',
    'N19117',
    'N8841L',
    'N190UW',
    'N65832'
]

In [13]:
data = []
for tail_number in tail_numbers:
    url = f"https://aeroapi.flightaware.com/aeroapi/flights/{tail_number}"
    headers = {
        'x-apikey': api_key
    }
    response = requests.get(url, headers=headers)
    data.extend([
        {
            "tail_number": flight["registration"],
            "origin": flight["origin"]["name"],
            "destination": flight["destination"]['name'],
            "departure_time": flight["actual_off"],
            "arrival_time": flight["actual_on"],
            "aircraft_type": flight["aircraft_type"],
            "status": flight["status"],
        }
        for flight in response.json()['flights'] if flight["actual_on"] is not None and flight["status"] != "result unknown"
    ])

In [14]:
# turn into a dataframe and show it
df = pd.DataFrame(data)

# create consistency with arrivals 
df['status'] = df['status'].replace('Arrived / Gate Arrival', 'Arrived')
df

,tail_number,origin,destination,departure_time,arrival_time,aircraft_type,status
0,N350XX,Fort Lauderdale Exec,Fort Lauderdale Intl,2025-02-24T14:28:13Z,2025-02-24T14:40:13Z,CL30,Arrived
1,N350XX,Chicago Exec,Fort Lauderdale Exec,2025-02-19T22:17:13Z,2025-02-20T00:56:48Z,CL30,Arrived
2,N350XX,Fort Lauderdale Exec,Chicago Exec,2025-02-19T18:53:03Z,2025-02-19T21:45:34Z,CL30,Arrived
3,N350XX,Witham Fld,Fort Lauderdale Exec,2025-02-17T05:39:46Z,2025-02-17T06:02:59Z,CL30,Arrived
4,N350XX,McClellan-Palomar,Witham Fld,2025-02-17T00:49:00Z,2025-02-17T04:55:29Z,CL30,Arrived
...,...,...,...,...,...,...,...
75,N65832,Denver Intl,San Diego Intl,2025-02-24T23:22:07Z,2025-02-25T01:19:00Z,B739,Arrived
76,N65832,Los Angeles Intl,Denver Intl,2025-02-23T18:00:38Z,2025-02-23T19:58:00Z,B739,Arrived / Delayed
77,N65832,Washington Dulles Intl,Los Angeles Intl,2025-02-22T03:34:27Z,2025-02-22T08:36:00Z,B739,Arrived
78,N65832,Cancun Int'l,Washington Dulles Intl,2025-02-21T22:28:54Z,2025-02-22T01:11:04Z,B739,Arrived


Specifically for Tom Cruise:

In [15]:
# separate into two dataframes: one for tom and one for the rest
tom_df = df[df['tail_number'] == 'N350XX']
rest_df = df[df['tail_number'] != 'N350XX']
tom_df

,tail_number,origin,destination,departure_time,arrival_time,aircraft_type,status
0,N350XX,Fort Lauderdale Exec,Fort Lauderdale Intl,2025-02-24T14:28:13Z,2025-02-24T14:40:13Z,CL30,Arrived
1,N350XX,Chicago Exec,Fort Lauderdale Exec,2025-02-19T22:17:13Z,2025-02-20T00:56:48Z,CL30,Arrived
2,N350XX,Fort Lauderdale Exec,Chicago Exec,2025-02-19T18:53:03Z,2025-02-19T21:45:34Z,CL30,Arrived
3,N350XX,Witham Fld,Fort Lauderdale Exec,2025-02-17T05:39:46Z,2025-02-17T06:02:59Z,CL30,Arrived
4,N350XX,McClellan-Palomar,Witham Fld,2025-02-17T00:49:00Z,2025-02-17T04:55:29Z,CL30,Arrived
5,N350XX,Van Nuys,McClellan-Palomar,2025-02-16T22:41:44Z,2025-02-16T23:05:37Z,CL30,Arrived
6,N350XX,McClellan-Palomar,Van Nuys,2025-02-16T00:47:13Z,2025-02-16T01:15:08Z,CL30,Arrived
7,N350XX,Palm Springs Intl,McClellan-Palomar,2025-02-15T23:31:24Z,2025-02-15T23:54:36Z,CL30,Arrived
8,N350XX,Teterboro,Palm Springs Intl,2025-02-15T17:09:25Z,2025-02-15T22:53:49Z,CL30,Arrived
9,N350XX,Washington Dulles Intl,Teterboro,2025-02-14T20:37:29Z,2025-02-14T21:34:10Z,CL30,Arrived


Finding airports that are common between Tom Cruise and the rest of the planes.

In [16]:
# retrieving all airports for each df
tom_airports = set(tom_df['origin'].tolist() + tom_df['destination'].tolist())
rest_airports = set(rest_df['origin'].tolist() + rest_df['destination'].tolist())

# find common between the two
shared = tom_airports.intersection(rest_airports)

# print shared airports
print(f"Common airports as an origin/destination:")
for airport in shared:
    print(f"- {airport}")

Common airports as an origin/destination:
- San Francisco Int'l
- Washington Dulles Intl
